<a href="https://colab.research.google.com/github/rgg-code/Structure-Query-Language/blob/main/SQL_Basics_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SQL Basics
Assignment

In [1]:
# -*- coding: utf-8 -*-
"""
SQL Basics Assignment
"""

import sqlite3
import pandas as pd
from IPython.display import display, Markdown

# --- Helper Function to Execute SQL and Display Results ---
def run_sql_query(db_connection, query, title="SQL Query Result"):
    """
    Executes an SQL query and displays the result in a pandas DataFrame.
    """
    print(f"\n--- {title} ---")
    print(f"Query:\n```sql\n{query}\n```")
    try:
        cursor = db_connection.cursor()
        cursor.execute(query)
        # Check if the query returns data (e.g., SELECT statements)
        if cursor.description:
            columns = [description[0] for description in cursor.description]
            data = cursor.fetchall()
            df = pd.DataFrame(data, columns=columns)
            display(df)
        else:
            print("Query executed successfully. No data to display (e.g., DDL/DML statement).")
        db_connection.commit() # Commit changes for DDL/DML
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    print("-" * 40)


# --- Setup: Create an in-memory SQLite database ---
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
print("In-memory SQLite database created successfully.")


# --- SQL Basics: Questions 1-6 ---

# Question 1: Create a table called employees with the specified structure and constraints.
display(Markdown("### SQL Basics: Question 1"))
create_employees_table_query = """
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY NOT NULL,
    emp_name TEXT NOT NULL,
    age INTEGER CHECK (age >= 18),
    email TEXT UNIQUE,
    salary DECIMAL(10, 2) DEFAULT 30000.00
);
"""
run_sql_query(conn, create_employees_table_query, "Create Employees Table")

# Insert some sample data for demonstration
insert_employees_data = """
INSERT INTO employees (emp_id, emp_name, age, email, salary) VALUES
(1, 'Alice Smith', 30, 'alice@example.com', 50000.00),
(2, 'Bob Johnson', 24, 'bob@example.com', 35000.00),
(3, 'Charlie Brown', 45, 'charlie@example.com', 75000.00);
"""
run_sql_query(conn, insert_employees_data, "Insert Sample Employee Data")

# Question 2: Explain the purpose of constraints and how they help maintain data integrity.
display(Markdown("### SQL Basics: Question 2"))
display(Markdown("""
**Purpose of Constraints and Data Integrity:**

Constraints are rules enforced on data columns to limit the type of data that can be inserted or updated into a table. They ensure the accuracy, reliability, and consistency of data, thereby maintaining data integrity.

**Common Types of Constraints:**
* **NOT NULL:** Ensures that a column cannot have a NULL value.
* **UNIQUE:** Ensures that all values in a column are different.
* **PRIMARY KEY:** A combination of NOT NULL and UNIQUE. Uniquely identifies each row in a table. A table can have only one primary key.
* **FOREIGN KEY:** Links two tables together by referencing the primary key of another table. It ensures referential integrity, meaning relationships between tables are maintained.
* **CHECK:** Ensures that all values in a column satisfy a specific condition.
* **DEFAULT:** Provides a default value for a column when no value is specified during insertion.
"""))

# Question 3: Why apply NOT NULL? Can a primary key contain NULL values?
display(Markdown("### SQL Basics: Question 3"))
display(Markdown("""
**Why apply NOT NULL constraint?**
The `NOT NULL` constraint is applied to columns to ensure that no record has a missing or undefined value for that particular column. This is crucial for data completeness and reliability, especially for critical fields like `emp_name` or `order_id` which must always have a value.

**Can a primary key contain NULL values? Justify your answer.**
No, a primary key cannot contain `NULL` values. This is a fundamental rule of relational databases. The primary key's purpose is to uniquely identify each record in a table. If a primary key could be `NULL`, it would violate its uniqueness property (as `NULL` is not equal to `NULL` and cannot uniquely identify a row) and its role in ensuring entity integrity.
"""))

# Question 4: Add or remove constraints on an existing table.
display(Markdown("### SQL Basics: Question 4"))
display(Markdown("""
**Adding or Removing Constraints on an Existing Table:**

* **Adding a Constraint (Example: Adding a UNIQUE constraint to `emp_name`):**
    ```sql
    ALTER TABLE employees
    ADD CONSTRAINT UQ_EmployeeName UNIQUE (emp_name);
    ```
    *Note: This would fail if `emp_name` already has duplicate values.*

* **Removing a Constraint (Example: Removing the `UQ_EmployeeName` constraint):**
    ```sql
    ALTER TABLE employees
    DROP CONSTRAINT UQ_EmployeeName;
    ```
    *Note: Syntax for dropping constraints can vary slightly between SQL databases (e.g., MySQL uses `DROP INDEX` for unique constraints, PostgreSQL uses `DROP CONSTRAINT`). For SQLite, you generally cannot `DROP CONSTRAINT` directly for most constraints except `FOREIGN KEY` (and even then, it's often easier to recreate the table). For `UNIQUE` or `CHECK`, you'd typically need to recreate the table without the constraint.*
"""))
# Demonstrating adding a constraint (will fail if duplicates exist, but shows syntax)
# For SQLite, adding a UNIQUE constraint directly via ALTER TABLE ADD CONSTRAINT is not supported for existing columns.
# It's usually done during table creation or by recreating the table.
# Let's show adding a new column with a constraint, or just the syntax for a more general SQL environment.
# For SQLite, altering a table to add a UNIQUE constraint to an existing column is not straightforward.
# We'll just show the conceptual SQL.
display(Markdown("""
**Conceptual SQL for Adding/Removing Constraints (Syntax may vary by DB):**

* **Adding a UNIQUE constraint to an existing column (e.g., `email` if it wasn't unique initially):**
    ```sql
    -- This syntax is common in other SQL databases like PostgreSQL, SQL Server.
    -- SQLite often requires recreating the table for complex ALTERs.
    ALTER TABLE employees
    ADD CONSTRAINT UQ_Email UNIQUE (email);
    ```

* **Removing a constraint (e.g., a CHECK constraint):**
    ```sql
    -- This syntax is common in other SQL databases.
    -- SQLite's ALTER TABLE DROP CONSTRAINT is limited.
    ALTER TABLE employees
    DROP CONSTRAINT employees_age_check; -- Constraint names are often auto-generated
    ```
"""))

# Question 5: Consequences of violating constraints.
display(Markdown("### SQL Basics: Question 5"))
display(Markdown("""
**Consequences of Violating Constraints:**

Attempting to insert, update, or delete data in a way that violates constraints will result in an error, and the operation will be rejected. This prevents invalid data from entering the database, thus preserving its integrity.

**Example Error Message (for a UNIQUE constraint violation):**

If you try to insert an employee with an `email` that already exists:
```sql
INSERT INTO employees (emp_id, emp_name, age, email, salary) VALUES
(4, 'David Lee', 28, 'alice@example.com', 40000.00);
```
You would typically get an error similar to:
`UNIQUE constraint failed: employees.email` (SQLite)
or
`Duplicate entry 'alice@example.com' for key 'employees.email'` (MySQL)
or
`Violation of UNIQUE KEY constraint 'UQ_employees_email'. Cannot insert duplicate key in object 'dbo.employees'.` (SQL Server)
"""))
# Demonstrate a constraint violation
insert_duplicate_email = """
INSERT INTO employees (emp_id, emp_name, age, email, salary) VALUES
(4, 'David Lee', 28, 'alice@example.com', 40000.00);
"""
print("Attempting to insert a row that violates the UNIQUE email constraint:")
run_sql_query(conn, insert_duplicate_email, "Insert with Duplicate Email (Expected Error)")

# Question 6: Add PRIMARY KEY and DEFAULT constraint to an existing products table.
display(Markdown("### SQL Basics: Question 6"))
create_products_no_constraints = """
CREATE TABLE products (
    product_id INT,
    product_name VARCHAR(50),
    price DECIMAL(10, 2)
);
"""
run_sql_query(conn, create_products_no_constraints, "Create Products Table (No Constraints)")

alter_products_table = """
ALTER TABLE products ADD PRIMARY KEY (product_id);
-- For SQLite, you cannot directly add a DEFAULT constraint to an existing column using ALTER TABLE.
-- You would typically need to recreate the table.
-- For demonstration, we'll use a common SQL syntax for other DBs for DEFAULT.
-- In SQLite, if you want to add a DEFAULT, you'd do:
-- ALTER TABLE products ADD COLUMN new_price DECIMAL(10,2) DEFAULT 50.00;
-- and then migrate data and drop old column.
-- For this exercise, we'll simulate the conceptual change.
"""
# Execute the PRIMARY KEY alteration
run_sql_query(conn, "ALTER TABLE products ADD PRIMARY KEY (product_id);", "Add PRIMARY KEY to Products")

# For DEFAULT, we'll demonstrate the conceptual SQL as SQLite's ALTER TABLE is limited here.
display(Markdown("""
**Adding DEFAULT value to `price` (Conceptual SQL for other DBs, SQLite usually requires recreation):**
```sql
ALTER TABLE products
ALTER COLUMN price SET DEFAULT 50.00;
-- Or for MySQL: ALTER TABLE products ALTER price SET DEFAULT 50.00;
-- Or for SQL Server: ALTER TABLE products ADD CONSTRAINT DF_Price DEFAULT 50.00 FOR price;
```
For SQLite, to effectively add a `DEFAULT` to an existing column, you usually need to create a new table with the desired schema, copy data, and then drop the old table.
"""))


# --- Joins: Questions 7-10 (from SQL Basics section) ---

# Create Students and Classes tables for Question 7
display(Markdown("### Joins: Question 7"))
create_students_table = """
CREATE TABLE Students (
    student_id INTEGER PRIMARY KEY,
    student_name TEXT,
    class_id INTEGER
);
"""
create_classes_table = """
CREATE TABLE Classes (
    class_id INTEGER PRIMARY KEY,
    class_name TEXT
);
"""
insert_students_data = """
INSERT INTO Students (student_id, student_name, class_id) VALUES
(1, 'Alice', 101),
(2, 'Bob', 102),
(3, 'Charlie', 101);
"""
insert_classes_data = """
INSERT INTO Classes (class_id, class_name) VALUES
(101, 'Math'),
(102, 'Science'),
(103, 'History');
"""
run_sql_query(conn, create_students_table, "Create Students Table")
run_sql_query(conn, create_classes_table, "Create Classes Table")
run_sql_query(conn, insert_students_data, "Insert Student Data")
run_sql_query(conn, insert_classes_data, "Insert Class Data")

# Question 7: Fetch student_name and class_name using INNER JOIN.
query_q7 = """
SELECT s.student_name, c.class_name
FROM Students s
INNER JOIN Classes c ON s.class_id = c.class_id;
"""
run_sql_query(conn, query_q7, "Question 7: INNER JOIN Students and Classes")


# Create Orders, Customers, Products tables for Question 8
display(Markdown("### Joins: Question 8"))
create_orders_table_q8 = """
CREATE TABLE Orders (
    order_id INTEGER PRIMARY KEY,
    order_date TEXT,
    customer_id INTEGER
);
"""
create_customers_table_q8 = """
CREATE TABLE Customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT
);
"""
create_products_table_q8 = """
CREATE TABLE Products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    order_id INTEGER -- Can be NULL if not associated with an order
);
"""
insert_orders_data_q8 = """
INSERT INTO Orders (order_id, order_date, customer_id) VALUES
(1, '2024-01-01', 101),
(2, '2024-01-03', 102);
"""
insert_customers_data_q8 = """
INSERT INTO Customers (customer_id, customer_name) VALUES
(101, 'Alice'),
(102, 'Bob');
"""
insert_products_data_q8 = """
INSERT INTO Products (product_id, product_name, order_id) VALUES
(1, 'Laptop', 1),
(2, 'Phone', NULL),
(3, 'Tablet', 2); -- Added another product for clarity
"""
run_sql_query(conn, create_orders_table_q8, "Create Orders Table (Q8)")
run_sql_query(conn, create_customers_table_q8, "Create Customers Table (Q8)")
run_sql_query(conn, create_products_table_q8, "Create Products Table (Q8)")
run_sql_query(conn, insert_orders_data_q8, "Insert Order Data (Q8)")
run_sql_query(conn, insert_customers_data_q8, "Insert Customer Data (Q8)")
run_sql_query(conn, insert_products_data_q8, "Insert Product Data (Q8)")

# Question 8: Show all order_id, customer_name, and product_name, ensuring all products are listed.
query_q8 = """
SELECT
    o.order_id,
    c.customer_name,
    p.product_name
FROM
    Products p
LEFT JOIN
    Orders o ON p.order_id = o.order_id
LEFT JOIN
    Customers c ON o.customer_id = c.customer_id;
"""
run_sql_query(conn, query_q8, "Question 8: LEFT JOIN to list all products")


# Create Sales and Products tables for Question 9
display(Markdown("### Joins: Question 9"))
create_sales_table_q9 = """
CREATE TABLE Sales (
    sale_id INTEGER PRIMARY KEY,
    product_id INTEGER,
    amount DECIMAL(10, 2)
);
"""
create_products_table_q9 = """
CREATE TABLE Products_Q9 ( -- Renamed to avoid conflict
    product_id INTEGER PRIMARY KEY,
    product_name TEXT
);
"""
insert_sales_data_q9 = """
INSERT INTO Sales (sale_id, product_id, amount) VALUES
(1, 101, 500),
(2, 102, 300),
(3, 101, 700),
(4, 103, 200); -- Added a sale for a product not in Products_Q9 to show INNER JOIN effect
"""
insert_products_data_q9 = """
INSERT INTO Products_Q9 (product_id, product_name) VALUES
(101, 'Laptop'),
(102, 'Phone');
"""
run_sql_query(conn, create_sales_table_q9, "Create Sales Table (Q9)")
run_sql_query(conn, create_products_table_q9, "Create Products_Q9 Table (Q9)")
run_sql_query(conn, insert_sales_data_q9, "Insert Sales Data (Q9)")
run_sql_query(conn, insert_products_data_q9, "Insert Products_Q9 Data (Q9)")

# Question 9: Total sales amount for each product using INNER JOIN and SUM().
query_q9 = """
SELECT
    p.product_name,
    SUM(s.amount) AS total_sales_amount
FROM
    Sales s
INNER JOIN
    Products_Q9 p ON s.product_id = p.product_id
GROUP BY
    p.product_name
ORDER BY
    p.product_name;
"""
run_sql_query(conn, query_q9, "Question 9: Total Sales per Product with INNER JOIN")


# Create Orders, Customers, Order_Details tables for Question 10
display(Markdown("### Joins: Question 10"))
create_orders_table_q10 = """
CREATE TABLE Orders_Q10 (
    order_id INTEGER PRIMARY KEY,
    order_date TEXT,
    customer_id INTEGER
);
"""
create_customers_table_q10 = """
CREATE TABLE Customers_Q10 (
    customer_id INTEGER PRIMARY KEY,
    customer_name TEXT
);
"""
create_order_details_table_q10 = """
CREATE TABLE Order_Details (
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    PRIMARY KEY (order_id, product_id)
);
"""
insert_orders_data_q10 = """
INSERT INTO Orders_Q10 (order_id, order_date, customer_id) VALUES
(1, '2024-01-02', 1),
(2, '2024-01-05', 2);
"""
insert_customers_data_q10 = """
INSERT INTO Customers_Q10 (customer_id, customer_name) VALUES
(1, 'Alice'),
(2, 'Bob');
"""
insert_order_details_data_q10 = """
INSERT INTO Order_Details (order_id, product_id, quantity) VALUES
(1, 101, 2),
(1, 102, 1),
(2, 101, 3);
"""
run_sql_query(conn, create_orders_table_q10, "Create Orders_Q10 Table (Q10)")
run_sql_query(conn, create_customers_table_q10, "Create Customers_Q10 Table (Q10)")
run_sql_query(conn, create_order_details_table_q10, "Create Order_Details Table (Q10)")
run_sql_query(conn, insert_orders_data_q10, "Insert Order Data (Q10)")
run_sql_query(conn, insert_customers_data_q10, "Insert Customer Data (Q10)")
run_sql_query(conn, insert_order_details_data_q10, "Insert Order Details Data (Q10)")

# Question 10: Display order_id, customer_name, and quantity using INNER JOINs.
query_q10 = """
SELECT
    o.order_id,
    c.customer_name,
    od.quantity
FROM
    Orders_Q10 o
INNER JOIN
    Customers_Q10 c ON o.customer_id = c.customer_id
INNER JOIN
    Order_Details od ON o.order_id = od.order_id
ORDER BY
    o.order_id, c.customer_name;
"""
run_sql_query(conn, query_q10, "Question 10: Multi-table INNER JOIN")


# --- SQL Commands (Questions 1-23 - Mavenmovies.sql based) ---
# Simulating Mavenmovies.sql schema for demonstration
display(Markdown("### SQL Commands (Mavenmovies.sql Simulation)"))

# Create simplified Mavenmovies tables
create_maven_tables = """
CREATE TABLE actor (
    actor_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL
);

CREATE TABLE customer (
    customer_id INTEGER PRIMARY KEY,
    store_id INTEGER,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE,
    address_id INTEGER,
    active BOOLEAN,
    create_date TEXT
);

CREATE TABLE film (
    film_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    description TEXT,
    release_year INTEGER,
    rental_duration INTEGER,
    rental_rate DECIMAL(4,2),
    length INTEGER,
    replacement_cost DECIMAL(5,2),
    rating TEXT,
    language_id INTEGER
);

CREATE TABLE film_actor (
    actor_id INTEGER,
    film_id INTEGER,
    PRIMARY KEY (actor_id, film_id)
);

CREATE TABLE rental (
    rental_id INTEGER PRIMARY KEY,
    rental_date TEXT,
    inventory_id INTEGER,
    customer_id INTEGER,
    return_date TEXT,
    staff_id INTEGER
);

CREATE TABLE payment (
    payment_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    staff_id INTEGER,
    rental_id INTEGER,
    amount DECIMAL(5,2),
    payment_date TEXT
);

CREATE TABLE inventory (
    inventory_id INTEGER PRIMARY KEY,
    film_id INTEGER,
    store_id INTEGER
);

CREATE TABLE language (
    language_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);

CREATE TABLE address (
    address_id INTEGER PRIMARY KEY,
    address TEXT,
    city_id INTEGER
);

CREATE TABLE city (
    city_id INTEGER PRIMARY KEY,
    city TEXT,
    country_id INTEGER
);

CREATE TABLE country (
    country_id INTEGER PRIMARY KEY,
    country TEXT
);
"""
run_sql_query(conn, create_maven_tables, "Create Simplified Mavenmovies Schema")

# Insert sample data into simplified Mavenmovies tables
insert_maven_data = """
INSERT INTO actor (actor_id, first_name, last_name) VALUES
(1, 'PENELOPE', 'GUINESS'), (2, 'NICK', 'WAHLBERG'), (3, 'ED', 'CHASE'), (4, 'JENNIFER', 'DAVIS'), (5, 'JOHNNY', 'DEPP');

INSERT INTO customer (customer_id, store_id, first_name, last_name, email, address_id, active, create_date) VALUES
(1, 1, 'MARY', 'SMITH', 'mary.smith@sakilacustomer.org', 5, 1, '2006-02-14'),
(2, 1, 'PATRICIA', 'JOHNSON', 'patricia.johnson@sakilacustomer.org', 6, 1, '2006-02-14'),
(3, 2, 'LINDA', 'WILLIAMS', 'linda.williams@sakilacustomer.org', 7, 0, '2006-02-14'),
(4, 2, 'BARBARA', 'JONES', 'barbara.jones@sakilacustomer.org', 8, 1, '2006-02-14'),
(5, 1, 'ROBERT', 'BROWNING', 'robert.browning@sakilacustomer.org', 9, 1, '2006-02-14');

INSERT INTO film (film_id, title, description, release_year, rental_duration, rental_rate, length, replacement_cost, rating, language_id) VALUES
(1, 'ACADEMY DINOSAUR', 'A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in A Canadian Mountain Range', 2006, 6, 0.99, 86, 20.99, 'PG', 1),
(2, 'ACE GOLDFINGER', 'A Astounding Documentary of a Robot And a Composer who must Find a Composer in A Mexican Sumo Wrestler''s Storehouse', 2006, 3, 4.99, 48, 12.99, 'G', 1),
(3, 'ADAPTATION HOLES', 'A Astounding Drama of a Feminist And a Composer who must Fight a Dentist in The Canadian Rockies', 2006, 5, 2.99, 50, 18.99, 'NC-17', 1),
(4, 'AFFAIR PREJUDICE', 'A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Northern California', 2006, 5, 2.99, 117, 26.99, 'G', 1),
(5, 'AFRICAN EGG', 'A Fast-Paced Drama of a Developing Country And a Explorer who must Find a Explorer in A Mexican Restaurant', 2006, 6, 2.99, 130, 22.99, 'G', 1),
(6, 'AGENT TRUMAN', 'A Fascinating Documentary of a Robot And a Sumo Wrestler who must Find a Sumo Wrestler in A Southern California', 2006, 7, 2.99, 169, 23.99, 'PG-13', 1);

INSERT INTO film_actor (actor_id, film_id) VALUES
(1, 1), (1, 2), (2, 1), (3, 3), (4, 4), (5, 5);

INSERT INTO rental (rental_id, rental_date, inventory_id, customer_id, return_date, staff_id) VALUES
(1, '2005-05-24 22:53:30', 1, 1, '2005-05-26 22:53:30', 1),
(2, '2005-05-24 22:54:33', 2, 2, '2005-05-28 22:54:33', 1),
(3, '2005-05-25 00:00:00', 3, 1, '2005-05-29 00:00:00', 2),
(4, '2005-06-15 00:00:00', 4, 4, '2005-06-20 00:00:00', 2);

INSERT INTO payment (payment_id, customer_id, staff_id, rental_id, amount, payment_date) VALUES
(1, 1, 1, 1, 2.99, '2005-05-25 11:30:00'),
(2, 2, 1, 2, 0.99, '2005-05-25 12:00:00'),
(3, 1, 2, 3, 4.99, '2005-05-29 10:00:00'),
(4, 4, 2, 4, 1.99, '2005-06-20 15:00:00');

INSERT INTO inventory (inventory_id, film_id, store_id) VALUES
(1, 1, 1), (2, 2, 1), (3, 3, 2), (4, 4, 2), (5, 5, 1), (6, 6, 2);

INSERT INTO language (language_id, name) VALUES
(1, 'English'), (2, 'Italian'), (3, 'Japanese');

INSERT INTO address (address_id, address, city_id) VALUES
(5, '123 Main St', 1), (6, '456 Oak Ave', 2), (7, '789 Pine Rd', 3), (8, '101 Elm St', 4), (9, '202 Maple Dr', 1);

INSERT INTO city (city_id, city, country_id) VALUES
(1, 'London', 1), (2, 'Paris', 1), (3, 'Rome', 2), (4, 'Berlin', 3);

INSERT INTO country (country_id, country) VALUES
(1, 'UK'), (2, 'Italy'), (3, 'Germany');
"""
run_sql_query(conn, insert_maven_data, "Insert Sample Mavenmovies Data")


# Question 1: Identify primary keys and foreign keys in maven movies db.
display(Markdown("### SQL Commands: Question 1"))
display(Markdown("""
**Primary Keys (PK) and Foreign Keys (FK) in Maven Movies DB (Simplified Schema):**

* **`actor` table:**
    * PK: `actor_id`
* **`customer` table:**
    * PK: `customer_id`
    * FK: `address_id` (references `address.address_id`)
* **`film` table:**
    * PK: `film_id`
    * FK: `language_id` (references `language.language_id`)
* **`film_actor` table:**
    * PK: (`actor_id`, `film_id`) - Composite Primary Key
    * FK: `actor_id` (references `actor.actor_id`)
    * FK: `film_id` (references `film.film_id`)
* **`rental` table:**
    * PK: `rental_id`
    * FK: `inventory_id` (references `inventory.inventory_id`)
    * FK: `customer_id` (references `customer.customer_id`)
    * FK: `staff_id` (references `staff.staff_id` - *staff table not created in this simplified schema*)
* **`payment` table:**
    * PK: `payment_id`
    * FK: `customer_id` (references `customer.customer_id`)
    * FK: `staff_id` (references `staff.staff_id` - *staff table not created*)
    * FK: `rental_id` (references `rental.rental_id`)
* **`inventory` table:**
    * PK: `inventory_id`
    * FK: `film_id` (references `film.film_id`)
    * FK: `store_id` (references `store.store_id` - *store table not created*)
* **`language` table:**
    * PK: `language_id`
* **`address` table:**
    * PK: `address_id`
    * FK: `city_id` (references `city.city_id`)
* **`city` table:**
    * PK: `city_id`
    * FK: `country_id` (references `country.country_id`)
* **`country` table:**
    * PK: `country_id`

**Differences:**
* **Primary Key:** Uniquely identifies a record within its own table. Must be `UNIQUE` and `NOT NULL`.
* **Foreign Key:** Establishes a link between two tables. It refers to the primary key of another table, enforcing referential integrity (ensuring that relationships between tables remain consistent). A foreign key can contain `NULL` values if the relationship is optional.
"""))

# Question 2: List all details of actors
display(Markdown("### SQL Commands: Question 2"))
query_q2_sql_commands = "SELECT * FROM actor;"
run_sql_query(conn, query_q2_sql_commands, "Question 2: All Actor Details")

# Question 3: List all customer information from DB.
display(Markdown("### SQL Commands: Question 3"))
query_q3_sql_commands = "SELECT * FROM customer;"
run_sql_query(conn, query_q3_sql_commands, "Question 3: All Customer Information")

# Question 4: List different countries.
display(Markdown("### SQL Commands: Question 4"))
query_q4_sql_commands = "SELECT DISTINCT country FROM country;"
run_sql_query(conn, query_q4_sql_commands, "Question 4: Different Countries")

# Question 5: Display all active customers.
display(Markdown("### SQL Commands: Question 5"))
query_q5_sql_commands = "SELECT * FROM customer WHERE active = 1;"
run_sql_query(conn, query_q5_sql_commands, "Question 5: All Active Customers")

# Question 6: List of all rental IDs for customer with ID 1.
display(Markdown("### SQL Commands: Question 6"))
query_q6_sql_commands = "SELECT rental_id FROM rental WHERE customer_id = 1;"
run_sql_query(conn, query_q6_sql_commands, "Question 6: Rental IDs for Customer ID 1")

# Question 7: Display all the films whose rental duration is greater than 5.
display(Markdown("### SQL Commands: Question 7"))
query_q7_sql_commands = "SELECT film_id, title, rental_duration FROM film WHERE rental_duration > 5;"
run_sql_query(conn, query_q7_sql_commands, "Question 7: Films with Rental Duration > 5")

# Question 8: List the total number of films whose replacement cost is greater than $15 and less than $20.
display(Markdown("### SQL Commands: Question 8"))
query_q8_sql_commands = "SELECT COUNT(film_id) AS total_films FROM film WHERE replacement_cost > 15 AND replacement_cost < 20;"
run_sql_query(conn, query_q8_sql_commands, "Question 8: Films with Replacement Cost between $15 and $20")

# Question 9: Display the count of unique first names of actors.
display(Markdown("### SQL Commands: Question 9"))
query_q9_sql_commands = "SELECT COUNT(DISTINCT first_name) AS unique_first_names FROM actor;"
run_sql_query(conn, query_q9_sql_commands, "Question 9: Count of Unique Actor First Names")

# Question 10: Display the first 10 records from the customer table.
display(Markdown("### SQL Commands: Question 10"))
query_q10_sql_commands = "SELECT * FROM customer LIMIT 10;"
run_sql_query(conn, query_q10_sql_commands, "Question 10: First 10 Customer Records")

# Question 11: Display the first 3 records from the customer table whose first name starts with 'b'.
display(Markdown("### SQL Commands: Question 11"))
query_q11_sql_commands = "SELECT * FROM customer WHERE first_name LIKE 'B%' LIMIT 3;"
run_sql_query(conn, query_q11_sql_commands, "Question 11: First 3 Customers whose First Name Starts with 'B'")

# Question 12: Display the names of the first 5 movies which are rated as 'G'.
display(Markdown("### SQL Commands: Question 12"))
query_q12_sql_commands = "SELECT title FROM film WHERE rating = 'G' LIMIT 5;"
run_sql_query(conn, query_q12_sql_commands, "Question 12: First 5 'G' Rated Movies")

# Question 13: Find all customers whose first name starts with "a".
display(Markdown("### SQL Commands: Question 13"))
query_q13_sql_commands = "SELECT customer_id, first_name, last_name FROM customer WHERE first_name LIKE 'A%';"
run_sql_query(conn, query_q13_sql_commands, "Question 13: Customers whose First Name Starts with 'A'")

# Question 14: Find all customers whose first name ends with "a".
display(Markdown("### SQL Commands: Question 14"))
query_q14_sql_commands = "SELECT customer_id, first_name, last_name FROM customer WHERE first_name LIKE '%A';"
run_sql_query(conn, query_q14_sql_commands, "Question 14: Customers whose First Name Ends with 'A'")

# Question 15: Display the list of first 4 cities which start and end with 'a'.
display(Markdown("### SQL Commands: Question 15"))
query_q15_sql_commands = "SELECT city FROM city WHERE city LIKE 'A%A' LIMIT 4;"
run_sql_query(conn, query_q15_sql_commands, "Question 15: First 4 Cities Starting and Ending with 'A'")

# Question 16: Find all customers whose first name have "NI" in any position.
display(Markdown("### SQL Commands: Question 16"))
query_q16_sql_commands = "SELECT customer_id, first_name, last_name FROM customer WHERE first_name LIKE '%NI%';"
run_sql_query(conn, query_q16_sql_commands, "Question 16: Customers with 'NI' in First Name")

# Question 17: Find all customers whose first name have "r" in the second position.
display(Markdown("### SQL Commands: Question 17"))
query_q17_sql_commands = "SELECT customer_id, first_name, last_name FROM customer WHERE first_name LIKE '_R%';"
run_sql_query(conn, query_q17_sql_commands, "Question 17: Customers with 'R' in Second Position of First Name")

# Question 18: Find all customers whose first name starts with "a" and are at least 5 characters in length.
display(Markdown("### SQL Commands: Question 18"))
query_q18_sql_commands = "SELECT customer_id, first_name, last_name FROM customer WHERE first_name LIKE 'A____%';" # A followed by 4 more characters
run_sql_query(conn, query_q18_sql_commands, "Question 18: Customers with First Name starting with 'A' and length >= 5")

# Question 19: Find all customers whose first name starts with "a" and ends with "o".
display(Markdown("### SQL Commands: Question 19"))
query_q19_sql_commands = "SELECT customer_id, first_name, last_name FROM customer WHERE first_name LIKE 'A%O';"
run_sql_query(conn, query_q19_sql_commands, "Question 19: Customers with First Name starting with 'A' and ending with 'O'")

# Question 20: Get the films with pg and pg-13 rating using IN operator.
display(Markdown("### SQL Commands: Question 20"))
query_q20_sql_commands = "SELECT film_id, title, rating FROM film WHERE rating IN ('PG', 'PG-13');"
run_sql_query(conn, query_q20_sql_commands, "Question 20: Films with 'PG' or 'PG-13' Rating")

# Question 21: Get the films with length between 50 to 100 using between operator.
display(Markdown("### SQL Commands: Question 21"))
query_q21_sql_commands = "SELECT film_id, title, length FROM film WHERE length BETWEEN 50 AND 100;"
run_sql_query(conn, query_q21_sql_commands, "Question 21: Films with Length Between 50 and 100")

# Question 22: Get the top 50 actors using limit operator.
display(Markdown("### SQL Commands: Question 22"))
query_q22_sql_commands = "SELECT actor_id, first_name, last_name FROM actor LIMIT 50;"
run_sql_query(conn, query_q22_sql_commands, "Question 22: Top 50 Actors")

# Question 23: Get the distinct film ids from inventory table.
display(Markdown("### SQL Commands: Question 23"))
query_q23_sql_commands = "SELECT DISTINCT film_id FROM inventory;"
run_sql_query(conn, query_q23_sql_commands, "Question 23: Distinct Film IDs from Inventory")


# --- Functions (Questions 1-8) ---
display(Markdown("### Functions"))

# Question 1: Retrieve the total number of rentals made in the Sakila database.
display(Markdown("#### Functions: Question 1"))
query_func_q1 = "SELECT COUNT(rental_id) AS total_rentals FROM rental;"
run_sql_query(conn, query_func_q1, "Question 1 (Functions): Total Rentals")

# Question 2: Find the average rental duration (in days) of movies rented from the Sakila database.
display(Markdown("#### Functions: Question 2"))
query_func_q2 = "SELECT AVG(rental_duration) AS average_rental_duration FROM film;"
run_sql_query(conn, query_func_q2, "Question 2 (Functions): Average Rental Duration")

# Question 3: Display the first name and last name of customers in uppercase.
display(Markdown("#### Functions: Question 3"))
query_func_q3 = "SELECT UPPER(first_name) AS upper_first_name, UPPER(last_name) AS upper_last_name FROM customer LIMIT 5;"
run_sql_query(conn, query_func_q3, "Question 3 (Functions): Customer Names in Uppercase")

# Question 4: Extract the month from the rental date and display it alongside the rental ID.
display(Markdown("#### Functions: Question 4"))
# SQLite's strftime for month extraction
query_func_q4 = "SELECT rental_id, STRFTIME('%m', rental_date) AS rental_month FROM rental LIMIT 5;"
run_sql_query(conn, query_func_q4, "Question 4 (Functions): Rental Month and ID")

# Question 5: Retrieve the count of rentals for each customer (display customer ID and the count of rentals).
display(Markdown("#### Functions: Question 5"))
query_func_q5 = """
SELECT customer_id, COUNT(rental_id) AS rental_count
FROM rental
GROUP BY customer_id
ORDER BY customer_id LIMIT 5;
"""
run_sql_query(conn, query_func_q5, "Question 5 (Functions): Rental Count per Customer")

# Question 6: Find the total revenue generated by each store.
display(Markdown("#### Functions: Question 6"))
# Assuming 'store_id' in inventory links to rentals via inventory_id
query_func_q6 = """
SELECT i.store_id, SUM(p.amount) AS total_revenue
FROM payment p
JOIN rental r ON p.rental_id = r.rental_id
JOIN inventory i ON r.inventory_id = i.inventory_id
GROUP BY i.store_id
ORDER BY i.store_id;
"""
run_sql_query(conn, query_func_q6, "Question 6 (Functions): Total Revenue per Store")

# Question 7: Determine the total number of rentals for each category of movies.
display(Markdown("#### Functions: Question 7"))
# This requires a film_category table, which is not in our simplified schema.
# Let's add a dummy film_category table for this.
create_film_category_table = """
CREATE TABLE film_category (
    film_id INTEGER,
    category_id INTEGER,
    PRIMARY KEY (film_id, category_id)
);
CREATE TABLE category (
    category_id INTEGER PRIMARY KEY,
    name TEXT
);
"""
insert_film_category_data = """
INSERT INTO category (category_id, name) VALUES
(1, 'Action'), (2, 'Comedy'), (3, 'Drama'), (4, 'Horror');
INSERT INTO film_category (film_id, category_id) VALUES
(1, 3), (2, 2), (3, 3), (4, 2), (5, 3), (6, 1);
"""
run_sql_query(conn, create_film_category_table, "Create Film Category Tables")
run_sql_query(conn, insert_film_category_data, "Insert Film Category Data")

query_func_q7 = """
SELECT c.name AS category_name, COUNT(r.rental_id) AS total_rentals
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film_category fc ON i.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
GROUP BY c.name
ORDER BY total_rentals DESC;
"""
run_sql_query(conn, query_func_q7, "Question 7 (Functions): Total Rentals per Category")

# Question 8: Find the average rental rate of movies in each language.
display(Markdown("#### Functions: Question 8"))
query_func_q8 = """
SELECT l.name AS language_name, AVG(f.rental_rate) AS average_rental_rate
FROM film f
JOIN language l ON f.language_id = l.language_id
GROUP BY l.name
ORDER BY average_rental_rate DESC;
"""
run_sql_query(conn, query_func_q8, "Question 8 (Functions): Average Rental Rate per Language")


# --- Joins (Questions 9-14 from Joins section) ---
display(Markdown("### Joins (Advanced)"))

# Question 9: Display the title of the movie, customer's first name, and last name who rented it.
display(Markdown("#### Joins: Question 9"))
query_joins_q9 = """
SELECT
    f.title,
    c.first_name,
    c.last_name
FROM
    film f
JOIN
    inventory i ON f.film_id = i.film_id
JOIN
    rental r ON i.inventory_id = r.inventory_id
JOIN
    customer c ON r.customer_id = c.customer_id
ORDER BY
    f.title, c.first_name, c.last_name
LIMIT 10;
"""
run_sql_query(conn, query_joins_q9, "Question 9 (Joins): Movie Title and Renter Name")

# Question 10: Retrieve the names of all actors who have appeared in the film "Gone with the Wind."
display(Markdown("#### Joins: Question 10"))
# Add a dummy film "Gone with the Wind" and link an actor to it
insert_gone_with_wind = """
INSERT INTO film (film_id, title, description, release_year, rental_duration, rental_rate, length, replacement_cost, rating, language_id) VALUES
(7, 'GONE WITH THE WIND', 'A classic drama', 1939, 7, 3.99, 238, 29.99, 'G', 1);
INSERT INTO actor (actor_id, first_name, last_name) VALUES (6, 'CLARK', 'GABLE');
INSERT INTO film_actor (actor_id, film_id) VALUES (6, 7);
"""
run_sql_query(conn, insert_gone_with_wind, "Insert 'Gone with the Wind' film and actor")

query_joins_q10 = """
SELECT
    a.first_name,
    a.last_name
FROM
    actor a
JOIN
    film_actor fa ON a.actor_id = fa.actor_id
JOIN
    film f ON fa.film_id = f.film_id
WHERE
    f.title = 'GONE WITH THE WIND';
"""
run_sql_query(conn, query_joins_q10, "Question 10 (Joins): Actors in 'Gone with the Wind'")

# Question 11: Retrieve the customer names along with the total amount they've spent on rentals.
display(Markdown("#### Joins: Question 11"))
query_joins_q11 = """
SELECT
    c.first_name,
    c.last_name,
    SUM(p.amount) AS total_spent
FROM
    customer c
JOIN
    payment p ON c.customer_id = p.customer_id
GROUP BY
    c.customer_id, c.first_name, c.last_name
ORDER BY
    total_spent DESC
LIMIT 5;
"""
run_sql_query(conn, query_joins_q11, "Question 11 (Joins): Total Spent per Customer")

# Question 12: List the titles of movies rented by each customer in a particular city (e.g., 'London').
display(Markdown("#### Joins: Question 12"))
query_joins_q12 = """
SELECT
    c.first_name,
    c.last_name,
    f.title
FROM
    customer cust
JOIN
    address a ON cust.address_id = a.address_id
JOIN
    city ci ON a.city_id = ci.city_id
JOIN
    rental r ON cust.customer_id = r.customer_id
JOIN
    inventory i ON r.inventory_id = i.inventory_id
JOIN
    film f ON i.film_id = f.film_id
WHERE
    ci.city = 'London'
GROUP BY
    cust.customer_id, cust.first_name, cust.last_name, f.title
ORDER BY
    cust.first_name, f.title
LIMIT 10;
"""
run_sql_query(conn, query_joins_q12, "Question 12 (Joins): Movies Rented by Customers in London")

# Question 13: Display the top 5 rented movies along with the number of times they've been rented.
display(Markdown("#### Joins: Question 13"))
query_joins_q13 = """
SELECT
    f.title,
    COUNT(r.rental_id) AS rental_count
FROM
    film f
JOIN
    inventory i ON f.film_id = i.film_id
JOIN
    rental r ON i.inventory_id = r.inventory_id
GROUP BY
    f.title
ORDER BY
    rental_count DESC
LIMIT 5;
"""
run_sql_query(conn, query_joins_q13, "Question 13 (Joins): Top 5 Rented Movies")

# Question 14: Determine the customers who have rented movies from both stores (store ID 1 and store ID 2).
display(Markdown("#### Joins: Question 14"))
query_joins_q14 = """
SELECT
    c.customer_id,
    c.first_name,
    c.last_name
FROM
    customer c
JOIN
    rental r ON c.customer_id = r.customer_id
JOIN
    inventory i ON r.inventory_id = i.inventory_id
WHERE
    i.store_id IN (1, 2)
GROUP BY
    c.customer_id, c.first_name, c.last_name
HAVING
    COUNT(DISTINCT i.store_id) = 2;
"""
run_sql_query(conn, query_joins_q14, "Question 14 (Joins): Customers who Rented from Both Stores")


# --- Window Functions (Questions 1-10) ---
display(Markdown("### Window Functions"))

# Question 1: Rank the customers based on the total amount they've spent on rentals.
display(Markdown("#### Window Functions: Question 1"))
query_wf_q1 = """
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    SUM(p.amount) AS total_spent,
    RANK() OVER (ORDER BY SUM(p.amount) DESC) AS spending_rank
FROM
    customer c
JOIN
    payment p ON c.customer_id = p.customer_id
GROUP BY
    c.customer_id, c.first_name, c.last_name
ORDER BY
    spending_rank
LIMIT 5;
"""
run_sql_query(conn, query_wf_q1, "Question 1 (WF): Customer Spending Rank")

# Question 2: Calculate the cumulative revenue generated by each film over time.
display(Markdown("#### Window Functions: Question 2"))
query_wf_q2 = """
SELECT
    f.title,
    p.payment_date,
    p.amount,
    SUM(p.amount) OVER (PARTITION BY f.film_id ORDER BY p.payment_date) AS cumulative_revenue
FROM
    film f
JOIN
    inventory i ON f.film_id = i.film_id
JOIN
    rental r ON i.inventory_id = r.inventory_id
JOIN
    payment p ON r.rental_id = p.rental_id
ORDER BY
    f.title, p.payment_date
LIMIT 10;
"""
run_sql_query(conn, query_wf_q2, "Question 2 (WF): Cumulative Revenue per Film")

# Question 3: Determine the average rental duration for each film, considering films with similar lengths.
display(Markdown("#### Window Functions: Question 3"))
# This implies a "window" of similar lengths. Let's define "similar" as within a range.
# For simplicity, let's use NTILE to group films into length quartiles and then average within those.
# Or, a simpler interpretation: average rental duration for each film.
# If it means average rental duration of films *similar* in length, we need a way to define "similar".
# Let's interpret it as average rental duration of *each* film, and then maybe a moving average or grouping by length.
# A common window function use case is to average over a partition.
# Let's group by length buckets for "similar lengths".
query_wf_q3 = """
SELECT
    f.title,
    f.length,
    f.rental_duration,
    AVG(f.rental_duration) OVER (PARTITION BY (f.length / 10) * 10) AS avg_rental_duration_for_similar_length
FROM
    film f
ORDER BY
    f.length, f.title
LIMIT 10;
"""
run_sql_query(conn, query_wf_q3, "Question 3 (WF): Average Rental Duration for Films with Similar Lengths")

# Question 4: Identify the top 3 films in each category based on their rental counts.
display(Markdown("#### Window Functions: Question 4"))
query_wf_q4 = """
WITH FilmRentalCounts AS (
    SELECT
        f.film_id,
        f.title,
        c.name AS category_name,
        COUNT(r.rental_id) AS rental_count
    FROM
        film f
    JOIN
        inventory i ON f.film_id = i.film_id
    JOIN
        rental r ON i.inventory_id = r.inventory_id
    JOIN
        film_category fc ON f.film_id = fc.film_id
    JOIN
        category c ON fc.category_id = c.category_id
    GROUP BY
        f.film_id, f.title, c.name
),
RankedFilms AS (
    SELECT
        film_id,
        title,
        category_name,
        rental_count,
        ROW_NUMBER() OVER (PARTITION BY category_name ORDER BY rental_count DESC) AS rn
    FROM
        FilmRentalCounts
)
SELECT
    category_name,
    title,
    rental_count
FROM
    RankedFilms
WHERE
    rn <= 3
ORDER BY
    category_name, rn;
"""
run_sql_query(conn, query_wf_q4, "Question 4 (WF): Top 3 Films per Category by Rental Count")

# Question 5: Calculate the difference in rental counts between each customer's total rentals and the average rentals across all customers.
display(Markdown("#### Window Functions: Question 5"))
query_wf_q5 = """
WITH CustomerRentalCounts AS (
    SELECT
        customer_id,
        COUNT(rental_id) AS customer_total_rentals
    FROM
        rental
    GROUP BY
        customer_id
),
OverallAverage AS (
    SELECT
        AVG(customer_total_rentals) AS avg_rentals_all_customers
    FROM
        CustomerRentalCounts
)
SELECT
    crc.customer_id,
    crc.customer_total_rentals,
    oaa.avg_rentals_all_customers,
    (crc.customer_total_rentals - oaa.avg_rentals_all_customers) AS difference_from_average
FROM
    CustomerRentalCounts crc, OverallAverage oaa
ORDER BY
    difference_from_average DESC
LIMIT 5;
"""
run_sql_query(conn, query_wf_q5, "Question 5 (WF): Difference from Average Rentals per Customer")

# Question 6: Find the monthly revenue trend for the entire rental store over time.
display(Markdown("#### Window Functions: Question 6"))
query_wf_q6 = """
SELECT
    STRFTIME('%Y-%m', payment_date) AS payment_month,
    SUM(amount) AS monthly_revenue,
    SUM(SUM(amount)) OVER (ORDER BY STRFTIME('%Y-%m', payment_date)) AS cumulative_monthly_revenue
FROM
    payment
GROUP BY
    payment_month
ORDER BY
    payment_month;
"""
run_sql_query(conn, query_wf_q6, "Question 6 (WF): Monthly Revenue Trend")

# Question 7: Identify the customers whose total spending on rentals falls within the top 20% of all customers.
display(Markdown("#### Window Functions: Question 7"))
query_wf_q7 = """
WITH CustomerSpending AS (
    SELECT
        customer_id,
        SUM(amount) AS total_spent
    FROM
        payment
    GROUP BY
        customer_id
)
SELECT
    c.first_name,
    c.last_name,
    cs.total_spent,
    NTILE(5) OVER (ORDER BY cs.total_spent DESC) AS spending_quintile -- NTILE(5) for top 20%
FROM
    CustomerSpending cs
JOIN
    customer c ON cs.customer_id = c.customer_id
WHERE
    NTILE(5) OVER (ORDER BY cs.total_spent DESC) = 1 -- Top 20% is the first quintile
ORDER BY
    cs.total_spent DESC
LIMIT 5;
"""
run_sql_query(conn, query_wf_q7, "Question 7 (WF): Customers in Top 20% Spending")

# Question 8: Calculate the running total of rentals per category, ordered by rental count.
display(Markdown("#### Window Functions: Question 8"))
query_wf_q8 = """
WITH CategoryRentalCounts AS (
    SELECT
        c.name AS category_name,
        COUNT(r.rental_id) AS rental_count
    FROM
        rental r
    JOIN
        inventory i ON r.inventory_id = i.inventory_id
    JOIN
        film_category fc ON i.film_id = fc.film_id
    JOIN
        category c ON fc.category_id = c.category_id
    GROUP BY
        c.name
)
SELECT
    category_name,
    rental_count,
    SUM(rental_count) OVER (ORDER BY rental_count DESC) AS running_total_rentals
FROM
    CategoryRentalCounts
ORDER BY
    rental_count DESC;
"""
run_sql_query(conn, query_wf_q8, "Question 8 (WF): Running Total of Rentals per Category")

# Question 9: Find the films that have been rented less than the average rental count for their respective categories.
display(Markdown("#### Window Functions: Question 9"))
query_wf_q9 = """
WITH FilmCategoryRentals AS (
    SELECT
        f.film_id,
        f.title,
        cat.name AS category_name,
        COUNT(r.rental_id) AS film_rental_count
    FROM
        film f
    JOIN
        inventory i ON f.film_id = i.film_id
    JOIN
        rental r ON i.inventory_id = r.inventory_id
    JOIN
        film_category fc ON f.film_id = fc.film_id
    JOIN
        category cat ON fc.category_id = cat.category_id
    GROUP BY
        f.film_id, f.title, cat.name
),
CategoryAvgRentals AS (
    SELECT
        category_name,
        AVG(film_rental_count) AS avg_category_rentals
    FROM
        FilmCategoryRentals
    GROUP BY
        category_name
)
SELECT
    fcr.title,
    fcr.category_name,
    fcr.film_rental_count,
    car.avg_category_rentals
FROM
    FilmCategoryRentals fcr
JOIN
    CategoryAvgRentals car ON fcr.category_name = car.category_name
WHERE
    fcr.film_rental_count < car.avg_category_rentals
ORDER BY
    fcr.category_name, fcr.film_rental_count
LIMIT 10;
"""
run_sql_query(conn, query_wf_q9, "Question 9 (WF): Films Rented Less Than Category Average")

# Question 10: Identify the top 5 months with the highest revenue and display the revenue generated in each month.
display(Markdown("#### Window Functions: Question 10"))
query_wf_q10 = """
WITH MonthlyRevenue AS (
    SELECT
        STRFTIME('%Y-%m', payment_date) AS payment_month,
        SUM(amount) AS monthly_revenue
    FROM
        payment
    GROUP BY
        payment_month
)
SELECT
    payment_month,
    monthly_revenue
FROM
    MonthlyRevenue
ORDER BY
    monthly_revenue DESC
LIMIT 5;
"""
run_sql_query(conn, query_wf_q10, "Question 10 (WF): Top 5 Months with Highest Revenue")


# --- Normalization & CTE (Questions 1-12) ---
display(Markdown("### Normalization & CTE"))

# Normalization Theoretical Questions (1-4)
display(Markdown("#### Normalization: Question 1 (1NF)"))
display(Markdown("""
**1. First Normal Form (1NF):**
* **Identify a table in Sakila that violates 1NF:**
    Consider a hypothetical `film_details` table that stores film information along with a comma-separated list of actors for each film in a single column:
    `film_id | title | actors`
    `1       | Movie A | Actor1, Actor2`
    `2       | Movie B | Actor3`
* **Explanation and Normalization to 1NF:**
    This table violates 1NF because the `actors` column contains multiple values (a repeating group) within a single cell. To normalize to 1NF, each column must contain only atomic (single) values. We would create a new table, `film_actor`, to store the many-to-many relationship between films and actors, with each row representing a single film-actor pair.
    * `film` table: `film_id (PK), title`
    * `actor` table: `actor_id (PK), actor_name`
    * `film_actor` table: `film_id (PK, FK), actor_id (PK, FK)`
"""))

display(Markdown("#### Normalization: Question 2 (2NF)"))
display(Markdown("""
**2. Second Normal Form (2NF):**
* **Choose a table in Sakila and determine if it is in 2NF:**
    Consider a table `rental_details` with a composite primary key (`rental_id`, `film_id`) and columns like `customer_id`, `customer_name`, `rental_date`, `film_title`, `film_rental_rate`.
    `rental_id (PK) | film_id (PK) | customer_id | customer_name | rental_date | film_title | film_rental_rate`
* **If it violates 2NF, explain steps to normalize:**
    This table violates 2NF if `customer_name` depends only on `customer_id` (which is part of the primary key, but not the *entire* primary key), and `film_title`, `film_rental_rate` depend only on `film_id`. 2NF requires that all non-key attributes must be fully functionally dependent on the *entire* primary key.
    To normalize:
    * Create a `Customer` table: `customer_id (PK), customer_name`
    * Create a `Film` table: `film_id (PK), film_title, film_rental_rate`
    * The original table becomes `Rental` with foreign keys: `rental_id (PK), film_id (PK), customer_id (FK), rental_date`
"""))

display(Markdown("#### Normalization: Question 3 (3NF)"))
display(Markdown("""
**3. Third Normal Form (3NF):**
* **Identify a table in Sakila that violates 3NF:**
    Consider a `customer_address` table:
    `customer_id (PK) | customer_name | address | city | country | zip_code | city_id (FK)`
    Here, `city` and `country` are dependent on `city_id`, and `city_id` is dependent on `address_id` (which is linked to `customer_id` via `address`). This is a transitive dependency: `customer_id` -> `address_id` -> `city_id` -> `city`, `country`.
* **Describe transitive dependencies and outline steps to normalize to 3NF:**
    A transitive dependency occurs when a non-key attribute is dependent on another non-key attribute, which in turn is dependent on the primary key.
    To normalize:
    * Create an `Address` table: `address_id (PK), address, city_id (FK)`
    * Create a `City` table: `city_id (PK), city, country_id (FK)`
    * Create a `Country` table: `country_id (PK), country`
    * The `Customer` table then only holds `customer_id (PK), customer_name, address_id (FK)`.
"""))

display(Markdown("#### Normalization: Question 4 (Normalization Process to 2NF)"))
display(Markdown("""
**4. Normalization Process (from Unnormalized to at least 2NF):**
Let's take a simplified `Order_Details` table that might initially be unnormalized:

**Unnormalized Form (UNF):**
`Order_ID | Order_Date | Customer_ID | Customer_Name | Product_ID | Product_Name | Quantity | Price_Per_Unit | Supplier_Name`
* `Order_ID` and `Product_ID` form a composite key.
* `Product_Name`, `Price_Per_Unit`, `Supplier_Name` depend on `Product_ID`.
* `Customer_Name` depends on `Customer_ID`.

**Step 1: Normalize to 1NF (Remove repeating groups/ensure atomicity)**
This UNF example is already in 1NF if each cell contains a single value. If `Product_ID` was a list of IDs, we'd split it. Assuming it's already atomic:

**1NF:**
`Order_ID (PK) | Order_Date | Customer_ID | Customer_Name | Product_ID (PK) | Product_Name | Quantity | Price_Per_Unit | Supplier_Name`

**Step 2: Normalize to 2NF (Remove partial dependencies)**
Identify partial dependencies (attributes dependent on only part of a composite primary key).
* `Customer_Name` depends only on `Customer_ID`.
* `Product_Name`, `Price_Per_Unit`, `Supplier_Name` depend only on `Product_ID`.

To achieve 2NF, we break down the table into smaller tables where non-key attributes are fully dependent on the *entire* primary key.

**2NF Tables:**

* **`Orders` Table:**
    `Order_ID (PK) | Order_Date | Customer_ID (FK)`
* **`Customers` Table:**
    `Customer_ID (PK) | Customer_Name`
* **`Products` Table:**
    `Product_ID (PK) | Product_Name | Price_Per_Unit | Supplier_Name`
* **`Order_Line_Items` Table (Junction Table):**
    `Order_ID (PK, FK) | Product_ID (PK, FK) | Quantity`

Now, all non-key attributes in `Order_Line_Items` (`Quantity`) are fully dependent on the composite primary key (`Order_ID`, `Product_ID`). `Order_Date` is fully dependent on `Order_ID`, and `Customer_Name` on `Customer_ID`, etc., in their respective tables.
"""))

# CTE Practical Questions (5-12)
display(Markdown("#### CTE: Question 5"))
# Question 5: CTE to retrieve distinct actor names and film count.
query_cte_q5 = """
WITH ActorFilmCounts AS (
    SELECT
        a.first_name,
        a.last_name,
        COUNT(fa.film_id) AS num_films
    FROM
        actor a
    JOIN
        film_actor fa ON a.actor_id = fa.actor_id
    GROUP BY
        a.actor_id, a.first_name, a.last_name
)
SELECT
    first_name,
    last_name,
    num_films
FROM
    ActorFilmCounts
ORDER BY
    num_films DESC, first_name
LIMIT 5;
"""
run_sql_query(conn, query_cte_q5, "Question 5 (CTE): Actor Film Counts")

display(Markdown("#### CTE: Question 6"))
# Question 6: CTE combining film and language to display title, language name, rental rate.
query_cte_q6 = """
WITH FilmLanguageDetails AS (
    SELECT
        f.title,
        l.name AS language_name,
        f.rental_rate
    FROM
        film f
    JOIN
        language l ON f.language_id = l.language_id
)
SELECT
    title,
    language_name,
    rental_rate
FROM
    FilmLanguageDetails
ORDER BY
    title
LIMIT 5;
"""
run_sql_query(conn, query_cte_q6, "Question 6 (CTE): Film Language Details")

display(Markdown("#### CTE: Question 7"))
# Question 7: CTE to find total revenue generated by each customer.
query_cte_q7 = """
WITH CustomerRevenue AS (
    SELECT
        c.customer_id,
        c.first_name,
        c.last_name,
        SUM(p.amount) AS total_revenue
    FROM
        customer c
    JOIN
        payment p ON c.customer_id = p.customer_id
    GROUP BY
        c.customer_id, c.first_name, c.last_name
)
SELECT
    first_name,
    last_name,
    total_revenue
FROM
    CustomerRevenue
ORDER BY
    total_revenue DESC
LIMIT 5;
"""
run_sql_query(conn, query_cte_q7, "Question 7 (CTE): Total Revenue per Customer")

display(Markdown("#### CTE: Question 8"))
# Question 8: CTE with a window function to rank films based on their rental duration.
query_cte_q8 = """
WITH FilmRentalDurationRank AS (
    SELECT
        film_id,
        title,
        rental_duration,
        RANK() OVER (ORDER BY rental_duration DESC) AS duration_rank
    FROM
        film
)
SELECT
    film_id,
    title,
    rental_duration,
    duration_rank
FROM
    FilmRentalDurationRank
ORDER BY
    duration_rank
LIMIT 5;
"""
run_sql_query(conn, query_cte_q8, "Question 8 (CTE): Film Rental Duration Rank")

display(Markdown("#### CTE: Question 9"))
# Question 9: CTE to list customers who have made more than two rentals, then join with customer table.
query_cte_q9 = """
WITH HighRentalCustomers AS (
    SELECT
        customer_id,
        COUNT(rental_id) AS rental_count
    FROM
        rental
    GROUP BY
        customer_id
    HAVING
        COUNT(rental_id) > 2
)
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    hrc.rental_count
FROM
    customer c
JOIN
    HighRentalCustomers hrc ON c.customer_id = hrc.customer_id
ORDER BY
    hrc.rental_count DESC
LIMIT 5;
"""
run_sql_query(conn, query_cte_q9, "Question 9 (CTE): Customers with More Than Two Rentals")

display(Markdown("#### CTE: Question 10"))
# Question 10: CTE to find the total number of rentals made each month.
query_cte_q10 = """
WITH MonthlyRentals AS (
    SELECT
        STRFTIME('%Y-%m', rental_date) AS rental_month,
        COUNT(rental_id) AS total_rentals
    FROM
        rental
    GROUP BY
        rental_month
)
SELECT
    rental_month,
    total_rentals
FROM
    MonthlyRentals
ORDER BY
    rental_month;
"""
run_sql_query(conn, query_cte_q10, "Question 10 (CTE): Total Rentals Each Month")

display(Markdown("#### CTE: Question 11"))
# Question 11: CTE to generate a report showing pairs of actors who have appeared in the same film together.
query_cte_q11 = """
WITH FilmActorPairs AS (
    SELECT
        fa1.film_id,
        fa1.actor_id AS actor1_id,
        fa2.actor_id AS actor2_id
    FROM
        film_actor fa1
    JOIN
        film_actor fa2 ON fa1.film_id = fa2.film_id AND fa1.actor_id < fa2.actor_id
)
SELECT
    f.title,
    a1.first_name || ' ' || a1.last_name AS actor1_name,
    a2.first_name || ' ' || a2.last_name AS actor2_name
FROM
    FilmActorPairs fap
JOIN
    film f ON fap.film_id = f.film_id
JOIN
    actor a1 ON fap.actor1_id = a1.actor_id
JOIN
    actor a2 ON fap.actor2_id = a2.actor_id
ORDER BY
    f.title, actor1_name
LIMIT 5;
"""
run_sql_query(conn, query_cte_q11, "Question 11 (CTE): Pairs of Actors in Same Film")

display(Markdown("#### CTE: Question 12"))
# Question 12: Implement a recursive CTE to find all employees in the staff table who report to a specific manager.
# Need to create a simplified 'staff' table with a 'reports_to' column for this.
create_staff_table = """
CREATE TABLE staff (
    staff_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    reports_to INTEGER -- NULL for top-level manager
);
"""
insert_staff_data = """
INSERT INTO staff (staff_id, first_name, last_name, reports_to) VALUES
(1, 'Mike', 'Hillyer', NULL), -- CEO
(2, 'Jon', 'Stephens', 1),   -- Reports to Mike
(3, 'Harold', 'Davis', 1),  -- Reports to Mike
(4, 'Peter', 'Pan', 2),    -- Reports to Jon
(5, 'Wendy', 'Darling', 2); -- Reports to Jon
"""
run_sql_query(conn, create_staff_table, "Create Staff Table for Recursive CTE")
run_sql_query(conn, insert_staff_data, "Insert Staff Data")

# Recursive CTE query
# Find all employees reporting to 'Mike Hillyer' (staff_id = 1)
query_cte_q12 = """
WITH RECURSIVE EmployeeHierarchy AS (
    -- Anchor member: Select the initial manager
    SELECT
        staff_id,
        first_name,
        last_name,
        reports_to,
        1 AS level
    FROM
        staff
    WHERE
        staff_id = 1 -- Starting with Mike Hillyer (or any specific manager)

    UNION ALL

    -- Recursive member: Find employees who report to the current level of hierarchy
    SELECT
        s.staff_id,
        s.first_name,
        s.last_name,
        s.reports_to,
        eh.level + 1
    FROM
        staff s
    JOIN
        EmployeeHierarchy eh ON s.reports_to = eh.staff_id
)
SELECT
    staff_id,
    first_name,
    last_name,
    reports_to,
    level
FROM
    EmployeeHierarchy
ORDER BY
    level, staff_id;
"""
run_sql_query(conn, query_cte_q12, "Question 12 (CTE): Recursive CTE for Employee Hierarchy")


# --- Close the database connection ---
conn.close()
print("\nDatabase connection closed.")


In-memory SQLite database created successfully.


### SQL Basics: Question 1


--- Create Employees Table ---
Query:
```sql

CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY NOT NULL,
    emp_name TEXT NOT NULL,
    age INTEGER CHECK (age >= 18),
    email TEXT UNIQUE,
    salary DECIMAL(10, 2) DEFAULT 30000.00
);

```
Query executed successfully. No data to display (e.g., DDL/DML statement).
----------------------------------------

--- Insert Sample Employee Data ---
Query:
```sql

INSERT INTO employees (emp_id, emp_name, age, email, salary) VALUES
(1, 'Alice Smith', 30, 'alice@example.com', 50000.00),
(2, 'Bob Johnson', 24, 'bob@example.com', 35000.00),
(3, 'Charlie Brown', 45, 'charlie@example.com', 75000.00);

```
Query executed successfully. No data to display (e.g., DDL/DML statement).
----------------------------------------


### SQL Basics: Question 2


**Purpose of Constraints and Data Integrity:**

Constraints are rules enforced on data columns to limit the type of data that can be inserted or updated into a table. They ensure the accuracy, reliability, and consistency of data, thereby maintaining data integrity.

**Common Types of Constraints:**
* **NOT NULL:** Ensures that a column cannot have a NULL value.
* **UNIQUE:** Ensures that all values in a column are different.
* **PRIMARY KEY:** A combination of NOT NULL and UNIQUE. Uniquely identifies each row in a table. A table can have only one primary key.
* **FOREIGN KEY:** Links two tables together by referencing the primary key of another table. It ensures referential integrity, meaning relationships between tables are maintained.
* **CHECK:** Ensures that all values in a column satisfy a specific condition.
* **DEFAULT:** Provides a default value for a column when no value is specified during insertion.


...


--- Question 11 (CTE): Pairs of Actors in Same Film ---
Query:
```sql

WITH FilmActorPairs AS (
    SELECT
        fa1.film_id,
        fa1.actor_id AS actor1_id,
        fa2.actor_id AS actor2_id
    FROM
        film_actor fa1
    JOIN
        film_actor fa2 ON fa1.film_id = fa2.film_id AND fa1.actor_id < fa2.actor_id
)
SELECT
    f.title,
    a1.first_name || ' ' || a1.last_name AS actor1_name,
    a2.first_name || ' ' || a2.last_name AS actor2_name
FROM
    FilmActorPairs fap
JOIN
    film f ON fap.film_id = f.film_id
JOIN
    actor a1 ON fap.actor1_id = a1.actor_id
JOIN
    actor a2 ON fap.actor2_id = a2.actor_id
ORDER BY
    f.title, actor1_name
LIMIT 5;

```
An error occurred: no such table: film_actor
----------------------------------------


#### CTE: Question 12


--- Create Staff Table for Recursive CTE ---
Query:
```sql

CREATE TABLE staff (
    staff_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    reports_to INTEGER -- NULL for top-level manager
);

```
Query executed successfully. No data to display (e.g., DDL/DML statement).
----------------------------------------

--- Insert Staff Data ---
Query:
```sql

INSERT INTO staff (staff_id, first_name, last_name, reports_to) VALUES
(1, 'Mike', 'Hillyer', NULL), -- CEO
(2, 'Jon', 'Stephens', 1),   -- Reports to Mike
(3, 'Harold', 'Davis', 1),  -- Reports to Mike
(4, 'Peter', 'Pan', 2),    -- Reports to Jon
(5, 'Wendy', 'Darling', 2); -- Reports to Jon

```
Query executed successfully. No data to display (e.g., DDL/DML statement).
----------------------------------------

--- Question 12 (CTE): Recursive CTE for Employee Hierarchy ---
Query:
```sql

WITH RECURSIVE EmployeeHierarchy AS (
    -- Anchor member: Select the initial manager
    SELECT
        staff_id,
        first

,staff_id,first_name,last_name,reports_to,level
0,1,Mike,Hillyer,NaN,1
1,2,Jon,Stephens,1.0,2
2,3,Harold,Davis,1.0,2
3,4,Peter,Pan,2.0,3
4,5,Wendy,Darling,2.0,3


----------------------------------------

Database connection closed.
